<a href="https://colab.research.google.com/github/danielx1611/ComputerVisionAssignments/blob/main/EstimateCameraPose/EstimateCameraPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimate the Pose of Cameras Using Planar Objects

## Dependency installations

In [ ]:
%pip install gradio numpy opencv-python matplotlib pytransform3d

In [ ]:
from functools import partial
import time
import gradio
import os
import json
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import pytransform3d.camera as pc
import pytransform3d.transformations as pt
import io
from cvxopt import matrix, printing
from pathlib import Path
from PIL import Image

def cv2_imshow(img):
    cv.imshow("img", img)
    time.sleep(1)
    cv.destroyAllWindows()

CONTENT_DIR = Path(".")

try:
    import google.colab.patches as patches
    print("in collab")
    cv2_imshow = patches.cv2_imshow
    CONTENT_DIR = Path("/content")

    if not Path("ComputerVisionAssignments").exists():
      !git clone https://github.com/danielx1611/ComputerVisionAssignments.git

    # Move test files to the Colab local directory (i.e., /content/)
    if not Path("chess_jpg").exists():
      !mkdir chess_jpg
    !cp -r ComputerVisionAssignments/PinholeCameraModel/chess_jpg/* ./chess_jpg

    if not Path("planar_jpg").exists():
      !mkdir planar_jpg
    !cp -r ComputerVisionAssignments/EstimateCameraPose/planar_jpg/* ./planar_jpg
    !cp ComputerVisionAssignments/EstimateCameraPose/ref_points.json .
except:
    pass

# using this for now
CONTENT_DIR = Path(".")
for img_path in CONTENT_DIR.glob("*.jpg"):
  print(img_path)

in collab


## Camera Calibration

In [ ]:
calibration_images_dir = CONTENT_DIR / "chess_jpg"
planar_images_dir = CONTENT_DIR / "planar_jpg"
output_dir = CONTENT_DIR / "output"
output_dir.mkdir(exist_ok=True)


class Calibration:
    @staticmethod
    def run():
        # termination criteria
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d point in real world space
        imgpoints = [] # 2d points in image plane.

        # images = glob.glob(calibration_images_dir + "/" + "*.jpg")

        images = list(calibration_images_dir.glob("*.jpg"))
        gray = None
        rgb_images = []
        for fname in images:
            img = cv.imread(fname)
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            # Find the chess board corners
            ret, corners = cv.findChessboardCorners(gray, (9,6), None)

            # If found, add object points, image points (after refining them)
            if ret == True:
                objpoints.append(objp)

                corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
                imgpoints.append(corners2)

                # Draw and display the corners
                cv.drawChessboardCorners(img, (9,6), corners2, ret)

                # store rgb image
                color_fixed = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                rgb_images.append(color_fixed)

        ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        h, w = img.shape[:2]
        calibration_data = {
            "K": mtx.tolist(),  # Intrinsic matrix
            "D": dist.tolist(),  # Distortion coefficients
            "R": [
                cv.Rodrigues(r)[0].tolist() for r in rvecs
            ],  # Convert rvec to rotation matrix
            "t": [t.tolist() for t in tvecs],  # Translation vectors
            "width": w,
            "height": h,
        }

        image_grid(rgb_images[:4], rows=2, cols=2, rgb=True, show_axes=False)

        with open("calibration.json", "w") as f:
                json.dump(calibration_data, f, indent=2)
        return "Calibration successful"

## Utils

In [ ]:
def image_grid(
    images,
    rows=None,
    cols=None,
    fill: bool = True,
    show_axes: bool = False,
    rgb: bool = True,
):
    """
    A util function for plotting a grid of images.

    Args:
        images: (N, H, W, 4) array of RGBA images
        rows: number of rows in the grid
        cols: number of columns in the grid
        fill: boolean indicating if the space between images should be filled
        show_axes: boolean indicating if the axes of the plots should be visible
        rgb: boolean, If True, only RGB channels are plotted.
            If False, only the alpha channel is plotted.

    Returns:
        None
    """
    if (rows is None) != (cols is None):
        raise ValueError("Specify either both rows and cols or neither.")

    if rows is None:
        rows = len(images)
        cols = 1

    gridspec_kw = {"wspace": 0.0, "hspace": 0.0} if fill else {}
    fig, axarr = plt.subplots(rows, cols, gridspec_kw=gridspec_kw, figsize=(15, 9))
    bleed = 0
    fig.subplots_adjust(left=bleed, bottom=bleed, right=(1 - bleed), top=(1 - bleed))

    for ax, im in zip(axarr.ravel(), images):
        if rgb:
            # only render RGB channels
            ax.imshow(im[..., :3])
        else:
            # only render Alpha channel
            ax.imshow(im[..., 3])
        if not show_axes:
            ax.set_axis_off()

    return fig

In [ ]:
class Utils:
    @staticmethod
    def project_points(W: np.ndarray,
                       Lambda: np.ndarray,
                       Rt: np.ndarray) -> np.ndarray:
        """ Helper function to project 3D points to 2D image plane"""

        # Convert points to homogeneous coordinates
        W_tilde = np.vstack((W, np.ones((1, W.shape[1]))))

        print(f"W_tilde = \n{W_tilde}\n")

        # Calculate perspective projection in homogeneous coordinates
        X_tilde = Lambda @ Rt @ W_tilde

        print(f"X_tilde =  \n{X_tilde}\n")

        # Apply perspective division to convert coordinates from homogeneous to Cartesian
        X_tilde /= X_tilde[2,:]
        # Convert coordinates from homogeneous to Cartesian
        X = X_tilde[0:2,:]

        return X

    @staticmethod
    def draw_coordinate_frame(image_points, img):
        #x0, y0 = int(image_points[0,0]), int(image_points[1,0])
        x0, y0 = image_points[:,0].astype(int)
        #print(str(type(x0)) + " " + str(type(y0)))
        cv.circle(img, (x0, y0), 48, (0, 0, 0), -1)

        #x1, y1 = int(image_points[0,1]), int(image_points[1,1])
        x1, y1 = image_points[:,1].astype(int)
        #print(str(type(x1)) + " " + str(type(y1)))
        img = cv.arrowedLine(img, (x0, y0), (x1, y1), (255, 0, 0), 16)

        #x2, y2 = int(image_points[0,2]), int(image_points[1,2])
        x2, y2 = image_points[:,2].astype(int)
        #print(str(type(x2)) + " " + str(type(y2)))
        img = cv.arrowedLine(img, (x0, y0), (x2, y2), (0, 255, 0), 16)

        #x3, y3 = int(image_points[0,3]), int(image_points[1,3])
        x3, y3 = image_points[:,3].astype(int)
        #print(str(type(x3)) + " " + str(type(y3)))
        img = cv.arrowedLine(img, (x0, y0), (x3, y3), (0, 0, 255), 16)

        plt.imshow(img)

    @staticmethod
    def build_Lambda(phi_x, phi_y, skew, delta_x, delta_y):
        """ Build the intrinsic camera matrix Lambda """
        Lambda = np.array([[phi_x,  skew, delta_x],
                           [    0, phi_y, delta_y],
                           [    0,     0,       1]])
        return Lambda

    @staticmethod
    def json_read(filename):
        # Parses the json file
        try:
            with open(os.path.abspath(filename)) as f:
                data = json.load(f)
            return data
        except:
            raise ValueError("Unable to read JSON {}".format(filename))

## Camera Pose and Position Extraction

In [ ]:
class Pose:
  def run_get_homography():
    # Make a list of input images of the rectangles.
    raw_images = [(img_path, cv.imread(img_path), json.loads(open(json_path, "r").read())) for img_path, json_path in zip(
        sorted(planar_images_dir.glob("*.jpg")),
        sorted(planar_images_dir.glob("*.json"))
    )]

    # Convert all images in the list from BGR to RGB to show correct colors.
    raw_images = [(path, cv.cvtColor(img, cv.COLOR_BGR2RGB), json) for path, img, json in raw_images]

    ########################################################################
    # In this example, the images are already scaled down and corrected
    # for lens distortion. When using images from your own camera, you
    # might need to scale them down and also correct for lens distortion.
    # To do that, uncomment the following steps and adapt the names of variables.
    ########################################################################

    # Undistort the images using the refined camera matrix (removes lens distortion)
    #undistorted_images = [cv.undistort(img, camera_matrix, distortion_coefficients, None, refined_camera_matrix) for img in scaled_images]

    # Display a sample of images
    before = image_grid(map(lambda i: i[1], raw_images[:4]), rows=2, cols=2, rgb=True, show_axes=False)

    # Object points in world coordinates.
    # The world coordinate frame and the object's coordinate frame are the same.
    ref_points = json.loads(open("ref_points.json", "r").read())

    calibration = json.loads(open("calibration.json", "r").read())

    mtx = np.array(calibration["K"])
    dist = np.array(calibration["D"])
    w = calibration["width"]
    h = calibration["height"]

    # new_mtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    # raw_images = [cv.undistort(img, mtx, dist, None, new_mtx) for img in raw_images]

    # x,y,w,h = roi
    # raw_images = [img[y:y+h, x:x+w] for img in raw_images]

    labelled_points = []

    for path, image, points in raw_images:
        for j in range(0,4):
            # Draw circles at the image features
            image = cv.circle(image,
                              (np.int32(points[j][0]),np.int32(points[j][1])),
                              32, (255,0,0), -1)

            # Draw text labels at the image features
            image = cv.putText(image,'{}'.format(j),
                              (np.int32(points[j][0]+15),np.int32(points[j][1]+10)),
                              cv.FONT_HERSHEY_SIMPLEX, 16, (0,0,0), 4)

        labelled_points.append(image)

    # Display a sample of images
    after = image_grid(labelled_points[:4], rows=2, cols=2, rgb=True, show_axes=False)

    # Find the homography matrix for each image
    homography_matrices = {}
    matches_mask = {}

    printing.options['dformat'] = '%.3f'
    printing.options['width'] = -1

    for idx, (path, _, points) in enumerate(raw_images):
        h, mask = cv.findHomography(np.array(ref_points), np.array(points), cv.RANSAC, 5.0)

        print(f"H({idx}) = ")
        print(matrix(h))
        homography_matrices[str(path)] = h.tolist()
        matches_mask[str(path)] = mask.ravel().tolist()

    with open("matrices.json", "w+") as f:
      f.write(json.dumps({
          "homography": homography_matrices,
          "matches": matches_mask
      }))

    return [before, after]

  @staticmethod
  def load_matrices():
    with open("matrices.json", "r") as f:
      data = json.load(f)

    data = {
        "homography": {
            k: np.array(v) for k, v in data["homography"].items()
        },
        "matches": {
            k: np.array(v) for k, v in data["matches"].items()
        }
    }

    return data["homography"], data["matches"]

  @staticmethod
  def load_calibration():
    with open("calibration.json", "r") as f:
      data = json.load(f)

    return np.array(data["K"]), np.array(data["D"]), np.array([data["width"], data["height"]])

  @staticmethod
  def extract_from_homography(homography, inverse_intrinsic_camera_matrix):
    """
    extractExtrinsicParametersFromHomography factorizes the homograpgy transformation to extract the camera pose,
                                              i.e., rotation matrix and translation vector.

    Input arguments:
    ------------------
        homography:    3x3 homography matrix
        inverse_intrinsic_camera_matrix: 3x3 intrinsics matrix

    return:
    -------
        Omega: 3x3 Rotation matrix,
        tau:   3x1 translation vector
    """

    # # Use openCV to estimate the camera's position and orientation
    # _, rotation, translation, _ = cv.decomposeHomographyMat(
    #     homography, camera_matrix)

    # # Add the rotation matrix and translation vector to the list of camera poses
    # camera_rotations.append(rotation)
    # camera_translations.append(translation)

    # Multiply the homography matrix by the inverse intrinsic camera matrix to eliminiate the effect
    # of the camera's intrinsic parameters
    homography_extrinsic = np.dot(inverse_intrinsic_camera_matrix, homography)

    # Estimate the first two columns of the rotation matrix by computing the SVD of the first two
    # columns of the homography matrix
    U, L, Vt = np.linalg.svd(homography_extrinsic[:, 0:2])
    new_L = np.array([[1, 0], [0, 1], [0, 0]])
    rotation_matrix = U @ new_L @ Vt

    # Estimate the third column of the rotation matrix by taking the cross product of the first two
    # columns
    third_column = np.cross(rotation_matrix[:, 0], rotation_matrix[:, 1])


    rotation_matrix = np.c_[rotation_matrix, third_column]

    # Verify that the rotation matrix is positive by checking its determinant
    if np.linalg.det(rotation_matrix) < 0:
        rotation_matrix[:, 2] *= -1

    # Estimate the scaling factor by taking the average of the scaling factors between the original
    # extrinsic homography matrix and the first two columns of the rotation matrix
    scaling_factor = np.sum(homography_extrinsic[:, 0:2] / rotation_matrix[:, 0:2]) / 6

    # Estimate the translation vector by dividing the third column of the extrinsic homography
    # matrix by the scaling factor
    translation = homography_extrinsic[:, 2] / scaling_factor

    return rotation_matrix, translation

  @staticmethod
  def extract_from_homography_cv(object_points, image_points, intrinsic_camera_matrix, dist_coeffs=None):
    """
    Estimate pose from 3D-2D correspondences using solvePnP.

    Parameters
    ----------
    object_points : np.ndarray (Nx3)
        Known 3D points in world coordinates
    image_points : np.ndarray (Nx2)
        Corresponding 2D points in image plane
    intrinsic_camera_matrix : np.ndarray (3x3)
        Camera intrinsic matrix
    dist_coeffs : np.ndarray (optional)
        Distortion coefficients

    Returns
    -------
    rotation_matrix : np.ndarray (3x3)
    translation_vector : np.ndarray (3x1)
    """

    if object_points.shape[0] != image_points.shape[0]:
        raise ValueError("object_points and image_points must have the same number of points")

    if object_points.shape[0] < 4:
        raise ValueError("At least 4 correspondences are required for solvePnP")


    success, rvec, tvec = cv.solvePnP(
        object_points,
        image_points,
        intrinsic_camera_matrix,
        dist_coeffs if dist_coeffs is not None else np.zeros((4, 1)),
        flags=0
    )

    if not success:
        raise RuntimeError("solvePnP failed to find a valid pose")

    rotation_matrix, _ = cv.Rodrigues(rvec)
    return rotation_matrix, tvec

  @staticmethod
  def plot_camera(
      rotation: np.ndarray, translation: np.ndarray, intrinsic_matrix: np.ndarray, dimensions: np.ndarray
  ):
      virtual_image_distance = 0.1

      # This is the camera coordinate frame
      # Camera pose, i.e., the matrix [R t] of extrinsic parameters
      Rt = np.block([rotation.T, -rotation.T @ translation.reshape((3,1))])

      # Convert Rt from 3x4 to a 4x4 transformation matrix
      Rt = np.vstack([Rt, [0, 0, 0, 1]])

      print(matrix(Rt))

      ax = pt.plot_transform(
          A2B=Rt,
          s=2
      )

      pc.plot_camera(
          ax,
          cam2world=Rt,
          M=intrinsic_matrix,
          sensor_size=dimensions,
          virtual_image_distance=virtual_image_distance,
      )

      return ax

  @staticmethod
  def draw_axes(
      img: np.ndarray,
      rotation: np.ndarray,
      translation: np.ndarray,
      mtx: np.ndarray,
      dist: np.ndarray
  ):
    fig = plt.figure()
    plt.imshow(img)

    scale_factor = 6
    W = scale_factor * np.array(
        [[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float64
    )

    rvec = cv.Rodrigues(rotation)[0]
    tvec = translation.reshape(3, 1)

    print(f"W = \n{W}\n")

    image_axes, jac = cv.projectPoints(W, rvec, tvec, mtx, dist)
    image_axes = image_axes.squeeze().T
    print(f"Projected image points = \n{image_axes}\n")

    Utils.draw_coordinate_frame(image_axes, img)

    return fig

  @staticmethod
  def run_homography_pose():
    homography, matches = Pose.load_matrices()
    mtx, dist, dimensions = Pose.load_calibration()

    # Calculate the inverse of the intrinsic camera matrix
    inv_mtx = np.linalg.inv(mtx)

    camera_rotations = []
    camera_translations = []

    axes = []

    for idx, (path, homography) in enumerate(homography.items()):
        # Extract extrinsic parameters from homography
        rotation_matrix, translation = Pose.extract_from_homography(homography, inv_mtx)

        print(rotation_matrix, translation)

        if idx < 4:
          axes.append(Pose.draw_axes(
              cv.imread(path),
              rotation_matrix,
              translation,
              mtx,
              dist
          ))

        # Add the rotation matrix and translation vector to the list of camera poses
        camera_rotations.append(rotation_matrix)
        camera_translations.append(translation)

    # Reference frame
    camera_rotations.append(np.eye(3))
    camera_translations.append(np.array([0, 0, 0]))

    fig = plt.figure()
    ax = None

    cam2world = pt.transform_from_pq([0, 0, 0, 0, 0, 0, 0])
    pt.plot_transform(
        ax,
        A2B=cam2world,
        s=3,
        # name="World"
    )


    # Print the camera poses
    for i, (rotation, translation) in enumerate(zip(camera_rotations, camera_translations)):
        ax = Pose.plot_camera(rotation, translation, mtx, dimensions)

    if ax is not None:
      ax.view_init(30, 70)
      ax.set_xlim(-100, 100)
      ax.set_ylim(-100, 100)
      ax.set_zlim(-100, 100)

    return [fig, *axes]


  @staticmethod
  def run_decompose_homography_pose():
      # Load saved homographies + calibration
      homography, matches = Pose.load_matrices()
      mtx, dist, dimensions = Pose.load_calibration()

      camera_rotations = []
      camera_translations = []

      axes = []

      ref_points = json.loads(open("ref_points.json", "r").read())
      ref_points = np.array([[*p[0], 0] for p in ref_points])
      print(ref_points)

      # Go through each image's homography
      for idx, (path, H) in enumerate(homography.items()):
          image = cv.imread(path)
          points = np.array(json.loads(open(path.replace(".jpg", ".json"), "r").read()), dtype=np.float32)
          print(points)
          # Calculate the inverse of the intrinsic camera matrix
          # Extract extrinsic parameters from homography
          rotation_matrix, translation = Pose.extract_from_homography_cv(ref_points, points, mtx, dist)

          axes.append(Pose.draw_axes(
              image,
              rotation_matrix,
              translation,
              mtx,
              dist
          ))

          # Add the rotation matrix and translation vector to the list of camera poses
          camera_rotations.append(rotation_matrix)
          camera_translations.append(translation)

      # Add reference world frame
      camera_rotations.append(np.eye(3))
      camera_translations.append(np.array([0, 0, 0]))

      # Create a new 3D figure for all cameras
      fig = plt.figure()
      ax = fig.add_subplot(111, projection="3d")

      # Plot world frame
      cam2world = pt.transform_from_pq([0, 0, 0, 0, 0, 0, 0])
      pt.plot_transform(ax, A2B=cam2world, s=3)

      # Plot each estimated camera pose
      for R, t in zip(camera_rotations, camera_translations):
          ax = Pose.plot_camera(R, t, mtx, dimensions)

      # Adjust visualization
      ax.view_init(30, 70)
      ax.set_xlim(-100, 100)
      ax.set_ylim(-100, 100)
      ax.set_zlim(-100, 100)

      return [fig, *axes]


## UI Tabs

In [ ]:
class CalibrationUI:
  @staticmethod
  def save_images(images):
    for image in images:
      destination = calibration_images_dir / Path(image.name).name
      with open(image.name, "rb") as src, open(destination, "wb") as dst:
        dst.write(src.read())
    return "Upload Successful!"

  @staticmethod
  def calibration_tab():
    with gradio.Tab("Calibration / Upload"):
      gradio.Markdown("### Calibration UI")
      upload = gradio.Files(file_types=[".jpeg", ".jpg"])
      upload_button = gradio.Button("Upload Images")
      upload_output = gradio.Textbox(label="Upload status")

      calibrate_button = gradio.Button("Run Calibration")

      upload_button.click(fn=CalibrationUI.save_images, inputs=upload, outputs=upload_output)
      calibrate_button.click(fn=Calibration.run, outputs=[gradio.Text()])

In [ ]:
class PoseEstimationUI:
  @staticmethod
  def save_images(images):
    for image in images:
      destination = planar_images_dir / Path(image.name).name
      with open(image.name, "rb") as src, open(destination, "wb") as dst:
        dst.write(src.read())

    return "Upload Successful!"

  @staticmethod
  def picker_tab():
    # --- Single-image point picker using notebook variable `my_image` ---
    import gradio as gr
    from PIL import Image, ImageDraw, ImageFont
    import numpy as np
    import threading, json, os, io
    font = ImageFont.load_default(size=48)
    # Data
    points_store = []
    app = None
    SELECTED_POINTS = None

    def _to_pil_from_numpy(arr: np.ndarray) -> Image.Image:
        arr = np.asarray(arr)
        # channel-first -> channel-last
        if arr.ndim == 3 and arr.shape[0] in (1,3,4) and arr.shape[-1] not in (1,3,4):
            arr = np.transpose(arr, (1,2,0))
        if np.issubdtype(arr.dtype, np.floating):
            # scale floats in [0,1] to [0,255]
            arr = (np.clip(arr, 0.0, 1.0) * 255.0).round().astype(np.uint8)
        elif arr.dtype != np.uint8:
            arr = np.clip(arr, 0, 255).astype(np.uint8)
        # Choose mode
        if arr.ndim == 2:
            return Image.fromarray(arr, mode="L")
        if arr.ndim == 3 and arr.shape[2] == 3:
            return Image.fromarray(arr, mode="RGB")
        if arr.ndim == 3 and arr.shape[2] == 4:
            return Image.fromarray(arr, mode="RGBA")
        if arr.ndim == 3 and arr.shape[2] == 1:
            return Image.fromarray(arr[:,:,0], mode="L")
        raise ValueError(f"Unsupported array shape: {arr.shape}")

    def _to_pil(img):
        if isinstance(img, Image.Image):
            return img
        if isinstance(img, np.ndarray):
            return _to_pil_from_numpy(img)
        raise gr.Error("Set `my_image` to a PIL image or NumPy array before launching.")

    def _draw_points(base_img: Image.Image, pts, radius=64):
        img = base_img.copy().convert("RGB")

        d = ImageDraw.Draw(img)
        for (idx, (x, y)) in enumerate(pts):
            d.ellipse([x-radius, y-radius, x+radius, y+radius], fill=(255,0,0))
            d.text((x,y), str(idx), font=font, fill=(0,0,0))
        return img

    # Prepare base image from notebook variable
    # if 'my_image' not in globals():
        # raise RuntimeError("Please define `my_image` (PIL image or NumPy array) before running this cell.")
    # base_pil = _to_pil(globals()['my_image'])


    def _refresh_numpy():
        """Return current preview (base + points) as numpy for Gradio."""
        return np.array(_draw_points(PoseEstimationUI.selected_image, points_store))

    def on_click(evt: gr.SelectData):
        # Get coordinates robustly
        x = y = None
        if hasattr(evt, "index") and evt.index is not None:
            try: x, y = evt.index
            except: pass
        if (x is None or y is None) and hasattr(evt, "x") and hasattr(evt, "y"):
            x, y = evt.x, evt.y
        if x is None or y is None:
            return gr.update(), json.dumps(points_store)

        # Clamp to image bounds
        w, h = PoseEstimationUI.selected_image.size
        x = int(max(0, min(w-1, x)))
        y = int(max(0, min(h-1, y)))

        points_store.append([x, y])
        return _refresh_numpy(), json.dumps(points_store)

    def undo_last():
        if points_store:
            points_store.pop()
        return _refresh_numpy(), json.dumps(points_store)

    def clear_points():
        points_store.clear()

        PoseEstimationUI.selected_image = None
        PoseEstimationUI.selected_image_path = None

        return [
            gr.update(visible=False), # Image
            gr.update(visible=True),  # Upload Area
            gr.update(visible=True),  # Upload Button
            ""                        # Points text
        ]

    def done_btn_click():
        """Save to notebook var `selected_points` and close the app."""

        name = PoseEstimationUI.selected_image_path.name.split(".")[0]
        destination = planar_images_dir / f"{name}.json"

        with open(destination, "w+") as f:
          f.write(json.dumps(points_store))

        return f"✅ Saved {len(points_store)}"

    def save_image(file):
      destination = planar_images_dir / Path(file.name).name
      with open(file.name, "rb") as src, open(destination, "wb") as dst:
        dst.write(src.read())

      PoseEstimationUI.selected_image_path = destination
      PoseEstimationUI.selected_image = _to_pil_from_numpy(cv.imread(destination))

      return [
        gr.update(visible=True, value=_refresh_numpy()),  # Image
        gr.update(visible=False, value=None),             # Upload Area
        gr.update(visible=False),                         # Upload Button
      ]

    #with gr.Tab("Pose Estimation (Point Picker)"):
    with gr.Blocks(title="Pose Estimation (Point Picker)") as demo:
        gr.Markdown("**Click on the image to add points.** Use Undo / Clear as needed, then press **Done**.")

        img = gr.Image(
          label="Image (click to add points)",
          type="numpy", interactive=True, sources=[],
          visible=False
        )

        upload = gr.File(file_types=[".jpg"])
        upload_button = gr.Button("Upload Images")


        with gr.Row():
            undo_btn = gr.Button("↩️ Undo")
            clear_btn = gr.Button("🧹 Clear")
            done_btn = gr.Button("✅ Done", variant="primary")
        pts_text = gr.Textbox(label="Points (JSON)", value="[]", interactive=False)
        status = gr.Markdown("")

        # One image used for both input and output
        img.select(on_click, inputs=None, outputs=[img, pts_text])
        upload_button.click(save_image, inputs=[upload], outputs=[img, upload, upload_button])
        undo_btn.click(lambda: undo_last(), outputs=[img, pts_text])
        clear_btn.click(clear_points, outputs=[img, upload, upload_button, pts_text])
        done_btn.click(done_btn_click, outputs=[status])

        app = demo.launch(inline=True, prevent_thread_lock=True, debug=True)
        return app

class HomographyUI:
  @staticmethod
  def homography_tab():
    with gradio.Tab("Homography Poses"):
      homography_get_button = gradio.Button("Calculate Homography Matrices")
      homography_get_button.click(fn=Pose.run_get_homography, outputs=[gradio.Plot() for i in range(2)])

      gradio.Markdown("### Homography Poses")

      homography_pose_button = gradio.Button("Run Homography -> Pose")
      homography_pose_button.click(fn=Pose.run_homography_pose, outputs=[gradio.Plot() for i in range(5)])

      cv_pose_button = gradio.Button("Run OpenCV Pose")
      cv_pose_button.click(fn=Pose.run_decompose_homography_pose, outputs=[gradio.Plot() for i in range(5)])


In [ ]:
class CentralUI:
  @staticmethod
  def run():
    with gradio.Blocks(title="Camera Calibration & Pose Tool") as ui:
        with gradio.Tabs():
            CalibrationUI.calibration_tab()
            #PoseEstimationUI.picker_tab()
            HomographyUI.homography_tab()

    ui.launch(share=False, inline=True, prevent_thread_lock=True, debug=True)

## Driver Code (Picker and Central UI App)

In [ ]:
PoseEstimationUI.picker_tab()

In [ ]:
CentralUI.run()